In [ ]:
import pandas as pd     
import numpy as np 
from tabulate import tabulate
from sportsipy.fb.schedule import Schedule
import holidays
from datetime import datetime, timedelta
import requests


In [ ]:
master = pd.read_parquet("./processed_data/3_categorical_master.parquet")
print(tabulate(master, headers="keys", tablefmt="psql"))


## Variables TimeSeries
Com es tracta d'un problema que involucra temporalitats, caldria dotar al set de dades de "lag features" i estadístiques de mitjanes que ajudin a establir relacions. Primer ordenarem, crearem 3 lag features i despres treurem tots els nulls que s'hagin generat en el procés

In [ ]:
#sort values
master.sort_values(by=["id_cliente", "fecha_pedido"], inplace=True)

master["fecha_pedido"] = pd.to_datetime(master["fecha_pedido"], format='%d.%m.%Y')

FECHA_INICIAL = master["fecha_pedido"].min() 
print(FECHA_INICIAL)

#3 lagged variables
bar_grouped = master.groupby("id_cliente")
master["fecha_ultimo_refill_1"] = bar_grouped["fecha_pedido"].shift(1)
master["fecha_ultimo_refill_2"] = bar_grouped["fecha_pedido"].shift(2)
master["fecha_ultimo_refill_3"] = bar_grouped["fecha_pedido"].shift(3)

#3 average variables
#THIS MUST BE DONE AFTER DATE PROCESSING

master.dropna(inplace=True)





## Dates & Timestamps
Les dates i timestamps són strings que dificilment el model (sobretot si es tracten d'arbres de decisió o xarxes neuronals) identificara correctament, caldria dividir en dies de la setmana, setmanes iso, anys per exemple. Per els timestamps guardarem l'hora i el minut ?

In [ ]:
date_columns = ["fecha_caducidad", "fecha_descarga", "fecha_pedido", "fecha_ultimo_refill_1", "fecha_ultimo_refill_2", "fecha_ultimo_refill_3"]
date_names = ["caducidad", "descarga", "pedido", "ultimo_refill_1", "ultimo_refill_2", "ultimo_refill_3"]

for date, name in zip(date_columns, date_names):
    master[date] = pd.to_datetime(master[date], format='%d.%m.%Y')
    master[f"fecha_{name}_discreta"] = (master[date] - FECHA_INICIAL).dt.days.astype(int)

print(tabulate(master, headers="keys", tablefmt="psql"))
    

In [ ]:
master.columns

A mode de resum, ara la nostre taula mestra conté informació de:

- proveidors i subproveidors
- clients
- vehicle que transporta
- centres de carrega de cervesa
- tipus de cervesa 
- dates de quan es va demanar/quan va arribar /quan caduca la comanda servida i les dates sequencials (a partir del primer ticket)
- litres demanats pel client i litres servits
- timestamps de quan s'inicia i acaba la descarrega
- temperatura i pressio dels tancs
- informacio logistica (id_albaran, id_lote_producto, id_orden_entrega)

Podriem enriquir encara més la taula posant informació de grans festivals, mundials de futbol, altres lligues rellevants d'altres esports i demñes

In [ ]:
master.to_parquet("./processed_data/4_timeserie_master.parquet", index=True)